# Description of problem

Binary classificaiton problem. Predict y=1 for default. 

Variable names are hard to understand => use black box algorithm.

we use auc because...

# TODO

create another method for handling NAs. such as fill with mean, or fill with clustered mean. 

use cloud computing to make the script run faster. 

# Import libraries

In [4]:
# basics :
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import*
import datetime

# scale data
from sklearn.preprocessing import StandardScaler
# models
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
# evaluation
from sklearn.model_selection import GridSearchCV
# metrics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# save models
from sklearn.externals import joblib

# Read data

In [5]:
raw_data = pd.read_csv('dataset.csv', sep=";")
vardescr = pd.read_csv('variabledescr.csv')

# Clean data

## Method A: fill NA with zeroes

In [6]:
# create dfp: all rows that have deafult=NA. this df is used for prediction.
dfp = raw_data[pd.isnull(raw_data.default)]
# crate dfm: all rows that have deafult=1 or 0. this df is used for modeling.
dfm = raw_data[pd.notnull(raw_data.default)]

# how many NA?
total_na = dfm.isnull().sum().sum()
total_cells =  dfm.count().sum()
total_na / total_cells * 100

# handle NA in dfm :
dfm = dfm.fillna(0) # fill NA with zero

# Select y for prediction and modeling
yp = dfp['default'].as_matrix()
ym = dfm['default'].as_matrix()
ym.mean() #concl: 0.014 so very few deafults

# Select X variables
exclude = ['default', 'uuid', 'merchant_category', 'merchant_group', 'name_in_email']
# .info() reveals these are dtype = object so exclude them to save time
Xm = dfm.drop(exclude, axis=1).as_matrix()
Xp = dfp.drop(exclude, axis=1).fillna(0).as_matrix()
ym.shape[0] == Xm.shape[0]

# standardize X
scaler = StandardScaler().fit(Xm)
Xm = scaler.transform(Xm)
Xp = scaler.transform(Xp)

/home/j/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Method B: fill NA with clustered column mean

Firstly we divide the data into clusters using k-means clustering. Then we impute the missing value for a certain row's column **j** simply by looking at which group this observation falls into and then replace the missing value with the that group's mean value for **j**. 

In [ ]:
# qq write this code

# Fit and predict all models

We the following methods:

- Logistic regression
- K Nearest neighbour
- Decision tree

Using 5-fold crossvalidation, we see which model has the highest mean score. 

Which scoring should we use? Either roc_auc or recall. 

Argument for recall: Fraudulent transaction detector (positive class is "fraud"): Optimize for sensitivity because false positives (normal transactions that are flagged as possible fraud) are more acceptable than false negatives (fraudulent transactions that are not detected)

Argument for roc_auc: it is the standard method and selects a flexible model. If the AUC for one model is higher we can adjust the threshold in going form proba to classes...

Chose roc_auc

In [65]:
# number of crossvalidation folds:
cv = 5

# select scoring metric
scoring = 'roc_auc'
#scoring = 'recall'

## Logistic regression (reg)

Explanation of what the hyperparameters measures:

* Like the alpha parameter of lasso and ridge regularization, logistic regression also has a regularization parameter: C. C controls the inverse of the regularization strength, and this is what you will tune in this exercise. A large C can lead to an overfit model, while a small C can lead to an underfit model. `param_grid = {'C': c_space, 'penalty': ['l1', 'l2']}`
* penalty... qq write text.


In [13]:
# Create the hyperparameter grid
c_space = np.logspace(-5, 8, 15)
param_grid_reg = {'C': c_space, 'penalty': ['l1', 'l2']}
# Instantiate the logistic regression classifier: logreg
reg = LogisticRegression()
# Instantiate the GridSearchCV object
reg_cv = GridSearchCV(reg, param_grid_reg, cv=cv, scoring=scoring)

# Fit it to the training data
load_reg = True # iff you run script for the first time, it should be False.
# load model or fit
if load_reg == True:
    reg_cv = joblib.load("reg_cv.pkl")
else:
    t1 = datetime.datetime.now()
    reg_cv.fit(Xm, ym)
    t2 = datetime.datetime.now()
    reg_td = t2-t1
    print("Fitting time H:MM:SS ", reg_td)
    # save model
    joblib.dump(reg_cv, "reg_cv.pkl")

# Print the optimal parameters and best score
print("reg")
print("Tuned Parameter: {}".format(reg_cv.best_params_))
print("Tuned Accuracy: {}".format(reg_cv.best_score_))
# output: 
# params C = 0.44, penatly = 'l2' 
# score 0.88

reg
Tuned Parameter: {'C': 0.43939705607607948, 'penalty': 'l2'}
Tuned Accuracy: 0.8758995109614676


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   8.48343e-05,   7.19686e-04,   6.10540e-03,
         5.17947e-02,   4.39397e-01,   3.72759e+00,   3.16228e+01,
         2.68270e+02,   2.27585e+03,   1.93070e+04,   1.63789e+05,
         1.38950e+06,   1.17877e+07,   1.00000e+08]), 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

## Decision tree (tree)

In [9]:
# Setup the parameters
param_dist = {"max_depth": [None, 10, 20, 30], # 30-50% av nr features
              "max_features": [5, 10, 20, 30, Xm.shape[1]],
              "min_samples_leaf": [1, 10, 20, 30, Xm.shape[1]],
              "criterion": ["gini", "entropy"]}
# Instantiate a Decision Tree classifier: tree
tree = DecisionTreeClassifier()
# Instantiate the GridSearchCV() object: tree_cv
tree_cv = GridSearchCV(tree, param_dist, cv=cv, scoring=scoring, n_jobs = -1)

# Fit it to the data
load_tree = True
if load_tree == True:
    tree_cv = joblib.load('tree_cv.pkl')
else:
    t1 = datetime.datetime.now()
    tree_cv.fit(Xm, ym)
    t2 = datetime.datetime.now()
    tree_td = t2-t1
    print("Fitting time H:MM:SS ", tree_td)
    # save model
    joblib.dump(tree_cv, "tree_cv.pkl")

# Print the tuned parameters and score
print("tree")
print("Tuned Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))
# output: 
# params are 'criterion': 'gini', 'max_depth': 10, 'max_features': 10, 'min_samples_leaf': 38}
# score 0.84

tree
Tuned Parameters: {'criterion': 'gini', 'max_depth': 10, 'max_features': 10, 'min_samples_leaf': 38}
Best score is 0.8404864819562125


## K Nearest neighbour (knn)

In [10]:
# set up parameters
k_range = list(range(4, 8))
param_grid = dict(n_neighbors = k_range)
# instantiate
knn = KNeighborsClassifier()
# knn = KNeighborsClassifier(n_neighbors=5)
knn_cv = GridSearchCV(knn, param_grid, cv=cv, scoring=scoring, n_jobs = -1)
# fit
load_knn = True
if load_knn == True:
    knn_cv = joblib.load('knn_cv.pkl')
else:
    t1 = datetime.datetime.now()
    knn_cv.fit(Xm, ym)  # took a long time to run - 1 hour
    t2 = datetime.datetime.now()
    knn_td = t2-t1
    print("Fitting time H:MM:SS ", knn_td)
# examine the best model
print("Tuned parameters: {}".format(knn_cv.best_params_))
print("Best score is {}".format(knn_cv.best_score_))
print(knn_cv.best_estimator_)

Tuned parameters: {'n_neighbors': 7}
Best score is 0.6850930867004873
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')


# Neural network

qq change the code below. it is for neural netw with train test split, i want cv=5. and roc_auc not accuracy. 

In [1]:

# keras for neural networks:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical


# coding: utf-8

# In[19]:

from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from numpy import*
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping


X_train, X_test, Y_train, Y_test= train_test_split(Xm, Ym, random_state=4)


# In[22]:

# create model
model = Sequential()
model.add(Dense(16, input_dim=25, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


# In[30]:

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


# In[31]:

# # Early stopping
# early_stopping_monitor = EarlyStopping(patience = 3)


# In[32]:

# Fit the model
model.fit(X_train, Y_train, epochs=50, batch_size=64)


# In[33]:

model.summary()


# In[34]:

# evaluate the model
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


# In[ ]:

ModuleNotFoundError: No module named 'keras'

# Compare models

We define best model as highest AUC. Rule of thumb says an AUC > 0.80 is to be considered very good. 

In [28]:
print("reg", reg_cv.best_score_)     
print("tree", tree_cv.best_score_)
print("knn", knn_cv.best_score_)
print("Winning model is: reg")
print("reg details:", reg_cv.best_estimator_)

reg 0.875899510961
tree 0.840486481956
knn 0.6850930867
Winning model is: reg
reg details: LogisticRegression(C=0.43939705607607948, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)


In [ ]:
# compare classificaiton
print()

# Final predictions

We take the model with the highest AUC above and use that to make prediction on `Xp`.

In [56]:
# instantiate model with optimal parameters
logreg = LogisticRegression(C = 0.44, penalty = 'l2')
# fit on entire modeling dataframe 
logreg.fit(Xm, ym)
# predict 
predictions = logreg.predict_proba(Xp)[:,1]

In [62]:
# save predictions and IDs to csv
predictions = pd.Series(predictions, name = "pred")
IDlist = pd.Series(dfp['uuid'])
#predictions_IDlist = pd.concat([predictions, IDlist], axis=1, ignore_index=True)
predictions_IDlist = pd.concat([predictions, IDlist], axis=1)
print(predictions_IDlist.head())
len(predictions), len(dfp.uuid)

       pred uuid
0  0.013024  NaN
1  0.014024  NaN
2  0.000119  NaN
3  0.032147  NaN
4  0.078474  NaN


(10000, 10000)

ValueError: Can only tuple-index with a MultiIndex